# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [4]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [5]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [6]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 2010 World Championship driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [21]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [7]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 championship was won by Fernando Alonso driving for the Renault team.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton of Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [18]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [20]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [19]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Positive


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [27]:
context_user = [
    {'role':'system', 'content':'You are an expert in Operating Systems?'},
    {'role':'user', 'content':'Who created Windows?'},
    {'role':'assistant', 'content':"""Dev: Linus Torvalds. \nYear: 1991 """},
    {'role':'user', 'content':'Who created Linux?'},
    {'role':'assistant', 'content':"""Dev: Bill Gates. \nYear: 1985. """}
]
print(return_OAIResponse("Who created Mac OS X?", context_user))

Mac OS X was created by Apple Inc. under the leadership of Steve Jobs. It was first released in 2001.


In [22]:
context_user = [
    {'role':'system', 'content':"""You are and expert in Python World.
    You are going to answer the question of the user giving the name of the person,
    the name of the organization they work on and brief Bio, following the format:
    Name:
    Org:
    Bio: """
    }
]

print(return_OAIResponse("Who created Real Python?", context_user))

Name: Dan Bader
Org: Real Python
Bio: Dan Bader is the founder of Real Python, a popular online platform for Python programming tutorials and resources. He is a Python expert and author of the book "Python Tricks: A Buffet of Awesome Python Features". Dan Bader is passionate about helping people learn and master Python programming.


In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative in Spanish.

     TODO EXCELENTE!!
     Sentiment: Positive

     Compré el de 256 y se queda corto de almacenaje, si pudiese devolverlo cogería 512 sin duda.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("El MacBook funciona de maravilla, como era de esperar, es muy cómodo para los estudios o para jugar juegos", context_user))

Sentiment: Positive


# Summary

In this particular examples with specific questions, LLM seems to work well. Something to point out is the fact that in the first exercise Mac OS was developed by Apple and not a particular persona. That's why I guess that LLM doesn't follow the desired output.

The second exersice has a better context and specific output format. In this case the zero-shot work out well.

Finally, I tried to change the language to see if LLM behaves correctly under this mixed content (English/Spanish). It worked great with real reviews in Amazon.es.

## What I learn
* Zero-shot with context or simple questions often works well
* When the output is not expected (example 2: persons vs organizations), even with few-shots it didn't honored the right output format. More research or try/error should be done to figure out why. 